In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Retinoblastoma"
cohort = "GSE59983"

# Input paths
in_trait_dir = "../../input/GEO/Retinoblastoma"
in_cohort_dir = "../../input/GEO/Retinoblastoma/GSE59983"

# Output paths
out_data_file = "../../output/preprocess/Retinoblastoma/GSE59983.csv"
out_gene_data_file = "../../output/preprocess/Retinoblastoma/gene_data/GSE59983.csv"
out_clinical_data_file = "../../output/preprocess/Retinoblastoma/clinical_data/GSE59983.csv"
json_path = "../../output/preprocess/Retinoblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiling of primary human retinoblastoma"
!Series_summary	"Background"
!Series_summary	"Retinoblastoma is a pediatric eye cancer associated with RB1 loss or MYCN amplification (RB1+/+MYCNA). There are controversies concerning the existence of molecular subtypes within RB1-/- retinoblastoma. To test whether these molecular subtypes exist, we performed molecular profiling."
!Series_summary	""
!Series_summary	"Methods"
!Series_summary	"Genome-wide mRNA expression profiling was performed on 76 primary human retinoblastomas. Expression profiling was complemented by genome-wide DNA profiling and clinical, histopathological, and ex vivo drug sensitivity data."
!Series_summary	""
!Series_summary	"Findings"
!Series_summary	"RNA and DNA profiling identified major variability between retinoblastomas. While gene expression differences between RB1+/+MYCNA and RB1-/- tumors seemed more dichotomous, differences within the RB1-/- tumors were grad

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyzing the output from previous steps
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Gene Expression Data Availability
is_gene_available = True  # The dataset contains gene expression data (Affymetrix human genome u133 plus 2.0 PM microarray)

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at Sample Characteristics Dictionary:
# {0: ['tissue: primary Rb tissue'], 1: ['uhc-subgroup: group 3', 'uhc-subgroup: group 1', 'uhc-subgroup: group 2']}

# For trait (Retinoblastoma):
# The dataset is about retinoblastoma samples, but there's no direct indicator of disease status in the sample characteristics
# All samples are from "primary Rb tissue" which means all are retinoblastoma cases
# Since all samples have the same value (all have retinoblastoma), this is not useful for association studies
trait_row = None  # No variable trait data available

# For age:
# No age information is provided in the sample characteristics
age_row = None

# For gender:
# No gender information is provided in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# Since all variables are not available in this dataset, we'll define placeholder conversion functions

def convert_trait(value):
    if value and ":" in value:
        val = value.split(":", 1)[1].strip()
        if "rb" in val.lower() or "retinoblastoma" in val.lower():
            return 1
        else:
            return 0
    return None

def convert_age(value):
    return None  # Not applicable

def convert_gender(value):
    return None  # Not applicable

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering and save information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step because trait_row is None (clinical data is not available for association studies)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print(gene_data.index[:20])


Index(['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at',
       '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at',
       '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at',
       '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at',
       '1552264_PM_a_at', '1552266_PM_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are probe IDs from an Affymetrix microarray platform, not standard human gene symbols.
# They need to be mapped to gene symbols for meaningful biological interpretation.
# The "_PM_" in the identifiers indicates "Perfect Match" probes from an Affymetrix array.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene mapping
probe_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get gene mapping dataframe by extracting the probe ID and gene symbol columns
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)

# Preview the mapping
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the mapped gene expression data
print("\nGene expression data preview (after mapping):")
print(gene_data.shape)
print(gene_data.index[:10])  # Print first 10 gene symbols


Gene mapping preview:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'Gene': ['DDR1', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}



Gene expression data preview (after mapping):
(18989, 76)
Index(['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AAA1', 'AAAS'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# From Step 2, we determined that trait data is not available (trait_row = None)
# Create a sample DataFrame that represents the dataset's structure
# We'll use this for the validation function
sample_df = pd.DataFrame({trait: [1] * 10}, index=normalized_gene_data.index[:10])

# Print diagnostic information
print("Dataset lacks trait variation for association studies.")
print("All samples are from primary retinoblastoma tissue, without control samples or disease severity indicators.")

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=False,
    is_biased=True,  # Dataset is biased (all samples have the same trait value)
    df=sample_df,  # Using sample dataframe for validation
    note="Dataset contains gene expression data from retinoblastoma samples but lacks trait variation for association studies. All samples are primary retinoblastoma tissue."
)

print(f"Data quality check completed. The dataset is not suitable for association studies due to lack of trait variation.")

Normalized gene data shape: (18622, 76)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Normalized gene data saved to ../../output/preprocess/Retinoblastoma/gene_data/GSE59983.csv
Dataset lacks trait variation for association studies.
All samples are from primary retinoblastoma tissue, without control samples or disease severity indicators.
Abnormality detected in the cohort: GSE59983. Preprocessing failed.
Data quality check completed. The dataset is not suitable for association studies due to lack of trait variation.
